In [ ]:
import pandas as pd
import geopandas as gp
import os
import numpy as np
import matplotlib
import fiona
import pathlib
import glob

In [ ]:
#### SET FILE PATHS ####

boundary_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/SP clipping area.shp"
clip_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Clipped"
raw_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw"
project_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected"

In [ ]:
# set project CRS
crs = 2926

# set boundary
boundary = gp.read_file(boundary_path)
boundary_project = boundary.to_crs(crs)
boundary_project.to_file(boundary_path)

In [ ]:
# project to project's crs and clip to boundary

files = glob.glob(raw_path + '/**/*.shp', recursive = True)

for f in files:
    file_name = os.path.basename(f)[:-4]
    print(f'processing {file_name}')
    gdf = gp.read_file(f)
    gdf_project = gdf.to_crs(crs)
    gdf_project.to_file(os.path.join(project_path, f'{file_name}_projected.shp'))
    print(os.path.join(project_path, f'{file_name}_projected.shp'))

    gdf_clip = gp.clip(gdf_project, boundary)
    gdf_clip.to_file(os.path.join(clip_path, f'{file_name}_clipped.shp'))
    print(os.path.join(clip_path, f'{file_name}_clipped.shp'))


STREET ANALYSIS

In [ ]:
# find average street width by functional class
#### TODO should this be within our zone or across the whole data set?
#### TODO change file path to one just processed

streets = gp.read_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Seattle_Streets/Seattle_Streets.shp")
streets = streets.to_crs(crs)


In [ ]:
## check how many streets are missing width values
#streets.SURFACEWID.value_counts()
#print('number of roads with no width: ', len(streets[streets.SURFACEWID == 0]))

streets['ARTCLASS'] = streets['ARTCLASS'].map(str)

## backfill data by average width for each functional class
# remove rows = 0 before finding average
streets_wid = streets[['ARTCLASS','SURFACEWID']]
streets_wid = streets_wid[streets_wid.SURFACEWID != 0]
streets_wid.SURFACEWID.value_counts()

## calculate average by arterial class and surface type
streets_wid_avg = streets_wid.groupby(['ARTCLASS']).mean()
streets_wid_avg.reset_index(inplace = True)
streets_wid_avg['ARTCLASS'] = streets_wid_avg['ARTCLASS'].map(str).str[0]

## histogram of artclass = 0 surface width
# IQR of not designated roads with a surface width ranges from 22-25. min = 8 and max = 84
#streets[streets['ARTCLASS']=='0.0'][streets['SURFACEWID']!= 0]['SURFACEWID'].describe()

In [ ]:
# bring in and process clipped data
#### TODO change file path
streets_clipped = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_clipped_projected.shp"

streets_clipped_gdf = gp.read_file(streets_clipped)
streets_clipped_gdf = streets_clipped_gdf.to_crs(crs)
streets_clipped_gdf['ARTCLASS'] = streets_clipped_gdf['ARTCLASS'].map(str)


In [ ]:
# backfill data in the CLIPPED AND PROJECTED FILE by average width for each functional class
# NOTE: arterial class == 9 has one feature with surface width == 0. there is no backfilled data for this road
streets_bfill = pd.merge(left = streets_clipped_gdf, right = streets_wid_avg, how = 'left', left_on = 'ARTCLASS', right_on = 'ARTCLASS')
streets_bfill[['OBJECTID','ARTCLASS','UNITDESC','ARTDESCRIP','SURFACEWID_x','SURFACEWID_y','geometry']]
streets_bfill['width_upd'] = np.where(streets_bfill['SURFACEWID_x'] == 0, streets_bfill['SURFACEWID_y'], streets_bfill['SURFACEWID_x'])

In [ ]:
# buffer by the surface width
streets_bfill['geometry'] = streets_bfill.buffer(distance = streets_bfill['width_upd'])
columns = ['OBJECTID','ARTCLASS','UNITDESC','STNAME_ORD','ARTDESCRIP','SURFACEWID_x','width_upd','geometry']


In [ ]:
# output buffers as polygons and as outline polylines
streets_bfill[columns].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/seattle_streets_buffer.shp")
streets_bfill.dissolve().boundary.to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/seattle_streets_buffer_outlines.shp")

In [ ]:
## looking at SND network to compare against seattle streets
SND = gp.read_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Raw/Street_Network_Database_(SND)-20230505T014240Z-001/Street_Network_Database_(SND)/Street_Network_Database_(SND).shp")

In [ ]:
# TODO: need to look up what the arterial class widths are
SND.columns
SND['ARTERIAL_C'].value_counts()

BUILDING ANALYSIS

In [ ]:
#### TODO UPDATE TO THE MORE RECENT FILE (https://seattlecitygis.maps.arcgis.com/home/item.html?id=c7265e8e78f14970bf375d9b2259e82a)

buildings = gp.read_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/buildings_2015_clipped_projected.shp")

In [ ]:
buildings.to_crs(crs).to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/buildings_2015_clipped_projected.shp")

In [ ]:
buildings.columns
# 5k buildings do not have building heights
# buildings['BP99_APEX'].value_counts()

# scatter plot of area x building apex for grouping and backfilling
buildings['SHAPE_Area'].describe()
buildings_scatter = buildings[buildings.BP99_APEX!=0]
buildings_scatter = buildings_scatter[buildings_scatter.SHAPE_Area!=0]
buildings_scatter.plot.scatter('SHAPE_Area','BP99_APEX')

In [ ]:
#TODO find default height neighborhood's one story buildings. there are so many 300'+ buildings??